<a href="https://colab.research.google.com/github/NizarMohd/RunOrWalk/blob/main/RunOrWalk2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **1. Preparation**



**Check Python version.**

In [32]:
!python -V

Python 3.7.12


**Check Cuda version.**

In [33]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


**Mount the Google drive.**

Mount your Google drive to store the dataset and the trained models.
Execute the cell below. Visit this [URL](https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code) to retrieve the authorization code and  enter the code at the prompt. 

In [34]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Create a project directory**

In [35]:
from os import path, chdir, getcwd, mkdir

# Choose a project name
projectName = "RunOrWalk"

# Project directory is in My Drive
projectDirectory = "/content/drive/My Drive/" + projectName

# Checks if cwd is in content folder
if getcwd() == "/content":
  # Makes project directory if it does not exist
  if not path.isdir(projectDirectory):
    mkdir(projectDirectory)
    print(f"Project {projectName} has been created!")
  else:
    print(f"Project {projectName} already exist!")
  # Changes to project directory
  chdir(projectDirectory)

print(f"The current working directory is {getcwd()}")

The current working directory is /content/drive/My Drive/RunOrWalk


**Install dependencies.**

In [36]:
pip install torch torchvision

In [37]:
pip install opencv-python

**Check GPU usage.**

In [38]:
# Check if runtime uses GPU
import torch

gpu_name = torch.cuda.get_device_name(0)

print("Using GPU", gpu_name)

Using GPU Tesla P100-PCIE-16GB


Load Dataset

In [39]:


# importing the libraries
import pandas as pd
import numpy as np

# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.utils.data import Dataset
from torch.utils.data import Sampler
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Dropout
from torch.optim import Adam, SGD
import tensorflow as tf
from torchvision import transforms
from torchvision.transforms import *


datasets = pd.read_csv('features.csv')

train_set, test_set = train_test_split(datasets, test_size = 0.1)
train_set = train_set.dropna()
test_set = test_set.dropna()
# # extract labels

# def extract_labels(df): 
#     x_train = [] 
#     y_train = [] 
 
#     i = 0 
#     while i + 1 < len(df): 
#         x_train_point = df[i: i + 1].filter(items=["min_ax", "min_ay", "min_az", "min_gx", "min_gy", "min_gz", "max_ax", "max_ay", "max_az", "max_gx", "max_gy", "max_gz", \
#           "avg_ax", "avg_ay", "avg_az", "avg_gx", "avg_gy", "avg_gz", "energy_ax", "energy_ay", "energy_az", "energy_gx", "energy_gy", "energy_gz", "label"]) 
#         x_train.append(x_train_point.to_numpy()) 
#         y_train.append(df['label'].iloc[i]) 
#         i = i + 1 
 
#     x_train = torch.from_numpy(np.array(x_train))
#     y_train = torch.from_numpy(np.array(y_train))
     
#     return x_train, y_train

# train_x, train_y = extract_labels(train_set)
# test_x, test_y = extract_labels(test_set)

# print(train_x.shape)
# print(train_y.shape)
# print(test_x.shape)
# print(test_y.shape)



class MyDataset(Dataset):
    
    
    def __init__(self, is_train=True):
      if is_train:
        self.data = train_set
      else:
        self.data = test_set
      print(f'loaded data with dataset size {len(self)}')
 
      
    def __getitem__(self, idx):
        classes = 2
        # print(self.data.iloc[idx])
        # sys.stdout.flush()
        item = self.data.iloc[idx] 
        image = item[0:24]
        label = item[24]
        image = torch.from_numpy(np.array(image))
        labels = []
        for i in range(classes):
            if i == label:
              labels.append(1)
            else:
              labels.append(0)
        labels = torch.IntTensor(labels)
        return image, labels

    def __len__(self):
        return len(self.data)


CNN Model

In [40]:
import torch.nn as nn
import torch.nn.functional as F
class Model(nn.Module):
  
  def __init__(self):
    super(Model, self).__init__()
    n_input = 24
    n_h1 = 12 
    n_h2 = 6
    n_output = 2
    self.h1 = nn.Linear(n_input, n_h1)
    self.h2 = nn.Linear(n_h1, n_h2)
    self.output = nn.Linear(n_h2, n_output)

    print(self)
  
  def forward(self, x):
    x = F.relu(self.h1(x))
    x = F.relu(self.h2(x))
    x = self.output(x)
    return x


# class Net(nn.Module):   
#     def __init__(self):
#         super(Net, self).__init__()
#         self.fc1 = nn.Linear(24,12)    # input layer
#         self.fc2 = nn.Linear(12, 6)            # hidden layer
#         self.out = nn.Linear(6, 2)
        

#     # Defining the forward pass    
#     def forward(self, x):
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.out(x)
#         return x


Train Model

In [41]:
from torch import optim
from sklearn.model_selection import StratifiedKFold

def accuracy_score(outputs, targets):
  sum = 0
  batch_size = targets.size(0)
  
  for i in range(batch_size):
    index_O = outputs.cpu().data.numpy()[i].argmax()
    index_T = targets.cpu().data.numpy()[i].argmax()

    if index_O == index_T:
      sum = sum + 1
  return sum / batch_size

def train():
  batch_size = 64
  num_epochs = 30
  num_workers = 2

  model = Model()
  model = model.cuda().float()
  
  loss_fn = nn.MSELoss()
  optimizer = optim.SGD(model.parameters(),lr = 0.001, momentum= 0.9, nesterov = True)
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=1, factor=0.3, verbose=True)

  train_set = MyDataset(is_train=True)
  validation_set = MyDataset(is_train=False)

  train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=batch_size, num_workers=num_workers, pin_memory=True, shuffle=True)
  validation_loader = torch.utils.data.DataLoader(
        validation_set, batch_size=batch_size, num_workers=num_workers, pin_memory=True, shuffle=False)

  for epoch in range(num_epochs):
    train_loss, valid_loss = [], []
    train_acc , val_acc = [], []
    # train
    model.train()

    # #quantize model
    # model.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
    # model = torch.quantization.prepare_qat(model, inplace=True)
 
    for i, (data, target) in enumerate(train_loader):
        data, target = data.cuda().float(), target.cuda().float()
        # clear gradients
        optimizer.zero_grad()

        # forward prop
        prediction = model(data)

        # loss calculation
        loss = loss_fn(prediction, target)

        #backward prop
        loss.backward()

        ## weight optimization
        optimizer.step()

        train_loss.append(loss.item())
        train_acc.append(accuracy_score(prediction, target))

        if i % 100 == 99:
                print(f'training: iteration {i} / {len(train_loader)}, avg train loss = {np.mean(train_loss):.4f}, '
                      f'train accuracy {np.mean(train_acc):.4f}')
    
    # eval
    model.eval()
    for i, (data, target) in enumerate(validation_loader):
        data, target = data.cuda().float(), target.cuda().float()
        prediction = model(data)
        loss = loss_fn(prediction, target)
        valid_loss.append(loss.item())
        val_acc.append(accuracy_score(prediction, target))

        if i % 100 == 99:
                print(f'validation: iteration {i} / {len(validation_loader)}, avg val loss = {np.mean(train_loss):.4f}, '
                      f'val accuracy {val_acc.avg:.4f}')
 
    # epoch summary
    print("Epoch:", epoch, "Train Loss:", np.mean(train_loss), "Train acc:", np.mean(train_acc), "Val Loss:", np.mean(valid_loss), "Val Acc:",  np.mean(val_acc))

    # lr scheduler
    scheduler.step(np.mean(valid_loss))

    # checkpoint
    if epoch % 2 == 1:
        torch.save(model.state_dict(), f'ckpt_e{epoch}.pth')
    
  torch.save(model.state_dict(), 'A0183398M-Model.pth')



  

if __name__ == "__main__":
    train()


Model(
  (h1): Linear(in_features=24, out_features=12, bias=True)
  (h2): Linear(in_features=12, out_features=6, bias=True)
  (output): Linear(in_features=6, out_features=2, bias=True)
)
loaded data with dataset size 16807
loaded data with dataset size 1869
training: iteration 99 / 263, avg train loss = 0.1392, train accuracy 0.8584
training: iteration 199 / 263, avg train loss = 0.1029, train accuracy 0.8988
Epoch: 0 Train Loss: 0.09003125973215802 Train acc: 0.9142704372623575 Val Loss: 0.044935589159528416 Val Acc: 0.9698317307692308
training: iteration 99 / 263, avg train loss = 0.0360, train accuracy 0.9762
training: iteration 199 / 263, avg train loss = 0.0316, train accuracy 0.9810
Epoch: 1 Train Loss: 0.030088414246846742 Train acc: 0.9824144486692015 Val Loss: 0.024768109867970147 Val Acc: 0.9911458333333333
training: iteration 99 / 263, avg train loss = 0.0226, train accuracy 0.9859
training: iteration 199 / 263, avg train loss = 0.0208, train accuracy 0.9895
Epoch: 2 Train L

Test Accuracy

In [42]:
from torch.nn.modules.container import ModuleList
import torch
from torch import nn
# Change [your_python_file] to the actual file containing the definitons of the classes


def test():
    my_model = Model()
    my_model = my_model.cuda()
    my_model.eval()

    my_model.load_state_dict(torch.load('A0183398M-Model.pth'))
    
    
    test_set = MyDataset(is_train=False)
    batch_size = 64


    # mp.set_start_method('spawn', force=True)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, \
                      shuffle=False)

    # test accuracy
    test_acc = []
    for i, (image,label) in enumerate(test_loader):
        image, label = image.cuda().float(), label.cuda().float()
        with torch.no_grad():
            prediction = my_model(image)
            acc = accuracy_score(prediction, label)
            test_acc.append(acc)

        if i % 10 == 9:
            print(f'test: iteration {i} / {len(test_loader)}, '
                  f'test accuracy {np.mean(test_acc):.4f}')

    print(f'evaluation finished, val acc {np.mean(test_acc):.4f}')

if __name__ == "__main__":
    test()

Model(
  (h1): Linear(in_features=24, out_features=12, bias=True)
  (h2): Linear(in_features=12, out_features=6, bias=True)
  (output): Linear(in_features=6, out_features=2, bias=True)
)
loaded data with dataset size 1869
test: iteration 9 / 30, test accuracy 0.9953
test: iteration 19 / 30, test accuracy 0.9961
test: iteration 29 / 30, test accuracy 0.9958
evaluation finished, val acc 0.9958


Extract weights and biases

In [43]:
from torch import nn
import torch
import torch.quantization.quantize_fx as quantize_fx
import copy
import csv

my_model = Model()
my_model = my_model.cuda()
my_model.eval()
my_model.load_state_dict(torch.load('A0183398M-Model.pth'))
f = open('/content/drive/MyDrive/RunOrWalk/weights.csv', 'w', encoding='UTF8', newline='')
writer = csv.writer(f)

w1 = []
w2 = []
w3 = []

b1 = []
b2 = []
b3 = []



for param_tensor in my_model.state_dict():
    # get each layer, print layer size
    print(param_tensor, "\t", my_model.state_dict()[param_tensor].size())
    print("\n")
    print(param_tensor, "\t", my_model.state_dict()[param_tensor])
    print("\n")

    # prepare min,max of each layer
    r_max = 0
    r_min = 0
    layer = my_model.state_dict()[param_tensor]

    
    # if param_tensor == 'h1.weight' or param_tensor == 'h2.weight' or param_tensor == 'output.weight':
    #   # writer.writerow("weights")
    #   print("weights")
    #   #layer is 2D array
    #   for row in layer:
    #     if r_max < max(row.cpu().numpy()):
    #       r_max = max(row.cpu().numpy())      
    #     if r_min > min(row.cpu().numpy()):
    #       r_min = min(row.cpu().numpy())
    # else:
    #   # layer is 1D array
    #   # writer.writerow("bias")
    #   print("bias")
    #   r_max = max(layer.cpu().numpy())
    #   r_min = min(layer.cpu().numpy())

    # s = 2 / (r_max - r_min)
    # c = -1 - s * r_min
    out = []

    
    r = 0
  

    if param_tensor == 'h1.weight' or param_tensor == 'h2.weight' or param_tensor == 'output.weight':
      #layer is 2D array
      for row in layer:
        r = r + 1
        for item in row:
          y = item.item() * (10**3)
          out.append(round(y))
          if param_tensor == 'h1.weight':
            w1.append(round(y))
          elif param_tensor == 'h2.weight':
            w2.append(round(y))
          else:
            w3.append(round(y))

      
    else:
      # layer is 1D array
      for item in layer:
          r = r + 1
          y = item.item() * (10**3)
          
          out.append(round(y))
          if param_tensor == 'h1.bias':
            b1.append(round(y))
          elif param_tensor == 'h2.bias':
            b2.append(round(y))
          else:
            b3.append(round(y))

    print(r)
    print(out)
    
    print("\n")


#  compute accuracy of quantized weights


test_set = MyDataset(is_train=False)
batch_size = 64


# mp.set_start_method('spawn', force=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, \
                  shuffle=False)

# test accuracy
test_acc = []

for k, (image,label) in enumerate(test_loader):
        
        acc = 0
        idx = 0
        preds = []
        for data in image:
          z1 = []
          z2 = []
          z3 = []
          # 24 attr, 64 data
          # compute Z1 = W1x + b1
          j = 0
          z = 0
          x = data.numpy()
          for i in range(0, 288):
            z = w1[i] * x[i%24] + z
            if i% 24 == 23:
              z = z + b1[j]
              j = j + 1
              if z < 0:
                z = 0
              z1. append(round(z/1000))
              z = 0
          

          # compute z2 = w2 * z1 + b2
          j = 0
          z = 0
          for i in range(0, 72):
            z = w2[i] * z1[i%12] + z
            if i% 12 == 11:
              z = z + b2[j]
              j = j + 1
              if z < 0:
                z = 0
              z2. append(round(z/1000))
              z = 0 
          

          # compute z3 = w3z2 + b2
          j = 0
          z = 0
          for i in range(0, 12):
            z = w3[i] * z2[i%6] + z
            if i% 6 == 5:
              z = z + b3[j]
              j = j + 1
              z3. append(z)
              z = 0
          
        
          pred = np.argmax(np.asarray(z3))
          preds.append(pred)

        sum = 0
        for i in range(label.size(0)):
          i_o = preds[i]
          i_t = np.argmax(label[i].numpy())
          if i_o == i_t:
            sum = sum + 1
        test_acc.append(acc/label.size(0))
        print(sum/label.size(0))

print(np.mean(test_acc))  
  

Model(
  (h1): Linear(in_features=24, out_features=12, bias=True)
  (h2): Linear(in_features=12, out_features=6, bias=True)
  (output): Linear(in_features=6, out_features=2, bias=True)
)
h1.weight 	 torch.Size([12, 24])


h1.weight 	 tensor([[ 1.1506e-01,  1.3441e-01, -2.6631e-02, -4.6613e-02, -1.2313e-01,
          5.8204e-02, -5.8045e-02, -2.8568e-02,  8.0751e-02, -1.5330e-01,
         -1.0259e-01,  1.4188e-01, -1.1627e-01, -1.2526e-01,  7.4990e-02,
          1.4217e-01, -1.9462e-01,  1.6400e-01, -1.2933e-01, -1.3758e-01,
          4.9228e-02, -1.0618e-01, -2.0152e-01, -2.6965e-02],
        [-1.6187e-01,  8.8895e-02,  2.0653e-01,  1.3889e-01,  7.6473e-02,
          1.4013e-01,  6.9450e-02,  1.1800e-01,  1.5020e-01, -5.6644e-02,
         -6.1516e-02, -2.8601e-02, -4.3682e-02, -2.1429e-01, -1.5407e-01,
         -1.4786e-01,  1.5407e-01, -1.7131e-01, -5.7463e-02,  1.1162e-01,
         -2.3938e-01, -7.6743e-02,  1.1809e-01,  1.4307e-01],
        [ 1.7483e-02,  1.3818e-01, -3.7618e-03, -5

In [44]:
!pip install -q hls4ml

In [45]:
import hls4ml

In [46]:
config = hls4ml.utils.config_from_pytorch_model(model)

NameError: ignored